In [ ]:
!pip install bs4 requests

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import os
from collections import deque
import json

In [ ]:
SAVE_DIR = r"../raw_data"
EXCLUDED_PATH = os.path.join(SAVE_DIR, "excluded_links.json")
DEPTH_LIMIT = 5

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [ ]:
SEED = ["https://vi.wikipedia.org/wiki/L%E1%BB%87_Quy%C3%AAn_(ca_s%C4%A9,_sinh_1981)", 
        "https://vi.wikipedia.org/wiki/Miu_L%C3%AA", 
        "https://vi.wikipedia.org/wiki/H%C3%B2a_Minzy",
        "https://vi.wikipedia.org/wiki/M%E1%BB%B9_Linh",
        "https://vi.wikipedia.org/wiki/Only_C",
        "https://vi.wikipedia.org/wiki/JustaTee",
        "https://vi.wikipedia.org/wiki/Ch%E1%BA%BF_Linh",
        "https://vi.wikipedia.org/wiki/%C4%90%C3%A0m_V%C4%A9nh_H%C6%B0ng",
        "https://vi.wikipedia.org/wiki/Mr._Siro"
        ]

In [ ]:
def remove_characters(text):
    if isinstance(text, str):
        # Bỏ cụm [sửa|sửa mã nguồn]
        cleaned = text.replace("[sửa|sửa mã nguồn]", "")
        # Xoá ngoặc và dấu cách/dấu ngoặc kép ở đầu & cuối
        cleaned = re.sub(r'^[\s\(\)\[\]\'"]+|[\s\(\)\[\]\'"]+$', '', cleaned)
        return cleaned.strip()
    return text


In [ ]:
def get_years(active_years):
    is_active = False
    if not active_years:
        return None, None

    if isinstance(active_years, str):
        active_years = [active_years]

    start_years = []
    end_years = []

    for period in active_years:
        if not period:
            continue
        p = period.strip()

        # Chuẩn hoá các loại dash thành hyphen thường
        p = re.sub(r'[–—−]', '-', p)

        # 🔹 Lấy tất cả năm và cả từ "nay"
        tokens = re.findall(r'\b(?:19|20)\d{2}\b|\b(?:nay|hiện tại|present|now)\b', p, re.IGNORECASE)

        # Nếu không có token nào, thử kiểm tra dạng đặc biệt "2015-"
        if not tokens:
            if re.search(r'\b(?:19|20)\d{2}\b\s*-\s*$', p):
                start = int(re.search(r'(?:19|20)\d{2}', p).group())
                start_years.append(start)
            continue

        # Xử lý token đầu tiên (năm bắt đầu)
        first = tokens[0]
        if re.match(r'(?:19|20)\d{2}', first):
            start_years.append(int(first))

        # Xử lý token cuối cùng (năm tan rã)
        last = tokens[-1]
        if re.match(r'(?:19|20)\d{2}', last):
            end_years.append(int(last))
        elif re.match(r'(nay|hiện tại|present|now)', last, re.IGNORECASE):
            is_active = True
            # nếu là 'nay' thì không có năm tan rã
            pass
        elif len(tokens) == 1:
            # chỉ có một năm, coi là hoạt động trong năm đó
            end_years.append(int(first))

    if not start_years:
        return None, None

    start = min(start_years)
    end = None if is_active else max(end_years)
    return start, end


In [ ]:
def load_excluded_links():
    if os.path.exists(EXCLUDED_PATH):
        with open(EXCLUDED_PATH, "r", encoding="utf-8") as f:
            return set(json.load(f))
    return set()

In [ ]:
def save_excluded_links(excluded_links):
    with open(EXCLUDED_PATH, "w", encoding="utf-8") as f:
        json.dump(sorted(list(excluded_links)), f, ensure_ascii=False, indent=2)

In [ ]:
def load_crawled_links():
    """Đọc danh sách link đã từng crawl."""
    if os.path.exists("crawled_links.json"):
        with open("crawled_links.json", "r", encoding="utf-8") as f:
            return set(json.load(f))
    return set()

def save_crawled_links(data):
    """Lưu danh sách link đã crawl."""
    with open("crawled_links.json", "w", encoding="utf-8") as f:
        json.dump(list(data), f, ensure_ascii=False, indent=2)


# ====== HÀM CHÍNH ======
def crawl_valid_links(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    stop_headings = ["Chú thích", "Tham khảo", "Liên kết ngoài"]

    content = soup.find('div', id='mw-content-text')
    if not content:
        print("⚠️ Không tìm thấy nội dung chính trong trang!")
        return []

    all_links = []

    for element in content.find_all(['p', 'ul', 'ol', 'div', 'h2', 'h3'], recursive=True):
        if element.name in ['h2', 'h3']:
            heading_text = element.get_text(strip=True)
            if any(stop in heading_text for stop in stop_headings):
                print(f"🛑 Dừng tại mục: {heading_text}")
                break

        for link in element.find_all('a', href=True):
            href = link['href']
            if href.startswith('/wiki/') and not any(x in href for x in [':', '#']):
                full_url = "https://vi.wikipedia.org" + href
                all_links.append(full_url)

    # Loại bỏ trùng lặp trong trang hiện tại
    all_links = list(dict.fromkeys(all_links))
    print(f"🔍 Tìm thấy {len(all_links)} đường dẫn hợp lệ trong trang chính.")

    excluded_links = load_excluded_links()
    crawled_links = load_crawled_links()

    print(f"📂 Bỏ qua {len(excluded_links)} link đã bị loại trước đó...")
    print(f"🧭 Bỏ qua {len(crawled_links)} link đã được crawl trước đó...")

    valid_links = []
    new_excluded = set()
    new_crawled = set()

    keywords = [
        "ca sĩ việt nam", "nam ca sĩ việt nam", "nữ ca sĩ việt nam",
        "ca sĩ gốc việt", "ca sĩ hải ngoại", "nhạc sĩ việt nam",
        "ban nhạc việt nam", "ban nhạc rock việt nam",
        "nhà sản xuất thu âm việt nam", "nhà sản xuất âm nhạc việt nam",
        "nhạc sĩ hòa âm phối khí việt nam", "rapper việt nam"
    ]
    keywords_lower = [k.lower() for k in keywords]

    for link in all_links:
        if link in excluded_links or link in crawled_links:
            print(f"⏩ Bỏ qua (đã loại hoặc đã crawl): {link}")
            continue

        try:
            sub_resp = requests.get(link, headers=headers, timeout=6)
            sub_soup = BeautifulSoup(sub_resp.text, 'html.parser')
            cat_div = sub_soup.find('div', id='mw-normal-catlinks')

            if cat_div:
                cat_text = cat_div.get_text(strip=True).lower()
                if any(k in cat_text for k in keywords_lower):
                    valid_links.append(link)
                    print(f"✅ Giữ lại: {link}")
                else:
                    print(f"❌ Loại bỏ: {link}")
                    new_excluded.add(link)
            else:
                print(f"⚠️ Không tìm thấy danh mục: {link}")
                new_excluded.add(link)

        except Exception as e:
            print(f"⚠️ Lỗi khi truy cập {link}: {e}")
            new_excluded.add(link)

        # Dù giữ hay loại, vẫn đánh dấu là đã crawl
        new_crawled.add(link)

    # Cập nhật dữ liệu
    excluded_links.update(new_excluded)
    crawled_links.update(new_crawled)
    save_excluded_links(excluded_links)
    save_crawled_links(crawled_links)

    print(f"💾 Đã lưu {len(new_crawled)} link mới vào danh sách đã crawl.")
    print(f"✅ Tổng số link hợp lệ mới: {len(valid_links)}")

    return valid_links

In [ ]:
def crawl_singer_award(soup):
    # Sử dụng set() để tự động chống trùng lặp là chính xác!
    awards = set()
    tables = soup.find_all("table", class_="wikitable")

    for table in tables:
        for row in table.find_all("tr"):
            cells = row.find_all(["td", "th"])
            if len(cells) < 2:
                continue  # Bỏ qua hàng không có đủ ô

            award_cell = cells[1]

            # Logic gốc của bạn: chỉ lấy những ô có thẻ <sup>
            if award_cell.find("sup"):
                text_parts = []
                # Lặp qua các nội dung con (gồm text và tag)
                for content in award_cell.contents:
                    if content.name == "sup":
                        break  # Dừng lại khi gặp <sup> đầu tiên
                    
                    # Lấy text, dù nó là chuỗi trần hay nằm trong tag khác (như <a>, <b>)
                    if isinstance(content, str):
                        text_parts.append(content)
                    elif hasattr(content, "get_text"):
                        text_parts.append(content.get_text())

                # 1. Ghép tất cả các phần text lại
                text = "".join(text_parts)
                
                # 2. Chuẩn hóa khoảng trắng:
                #    Biến "Giải thưởng\n   Ca sĩ" thành "Giải thưởng Ca sĩ"
                text = " ".join(text.split())

                if text:
                    awards.add(text)  # set sẽ tự lo việc chống trùng lặp

    return list(awards)


In [ ]:
def load_crawled_links():
    """Đọc danh sách link đã từng crawl."""
    if os.path.exists("crawled_links.json"):
        with open("crawled_links.json", "r", encoding="utf-8") as f:
            return set(json.load(f))
    return set()

def save_crawled_links(data):
    """Lưu danh sách link đã crawl."""
    with open("crawled_links.json", "w", encoding="utf-8") as f:
        json.dump(list(data), f, ensure_ascii=False, indent=2)

In [ ]:
def crawl_valid_links(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    stop_headings = ["Chú thích", "Tham khảo", "Liên kết ngoài"]
    content = soup.find('div', id='mw-content-text')
    if not content:
        print("⚠️ Không tìm thấy nội dung chính trong trang!")
        return []

    all_links = []
    for element in content.find_all(['p', 'ul', 'ol', 'div', 'h2', 'h3'], recursive=True):
        if element.name in ['h2', 'h3']:
            heading_text = element.get_text(strip=True)
            if any(stop in heading_text for stop in stop_headings):
                print(f"🛑 Dừng tại mục: {heading_text}")
                break
        for link in element.find_all('a', href=True):
            href = link['href']
            if href.startswith('/wiki/') and not any(x in href for x in [':', '#']):
                all_links.append("https://vi.wikipedia.org" + href)

    all_links = list(dict.fromkeys(all_links))
    print(f"🔍 Tìm thấy {len(all_links)} đường dẫn hợp lệ trong trang chính.")

    excluded_links = load_excluded_links()
    crawled_links = load_crawled_links()

    print(f"📂 Bỏ qua {len(excluded_links)} link đã bị loại trước đó...")
    print(f"🧭 Bỏ qua {len(crawled_links)} link đã được crawl trước đó...")

    valid_links_new = []
    new_excluded = set()
    new_crawled = set()

    keywords = [
        "ca sĩ việt nam", "nam ca sĩ việt nam", "nữ ca sĩ việt nam",
        "ca sĩ gốc việt", "ca sĩ hải ngoại", "nhạc sĩ việt nam",
        "ban nhạc việt nam", "ban nhạc rock việt nam",
        "nhà sản xuất thu âm việt nam", "nhà sản xuất âm nhạc việt nam",
        "nhạc sĩ hòa âm phối khí việt nam", "rapper việt nam"
    ]
    keywords_lower = [k.lower() for k in keywords]

    for link in all_links:
        if link in excluded_links or link in crawled_links:
            print(f"⏩ Bỏ qua (đã loại hoặc đã crawl): {link}")
            continue

        try:
            sub_resp = requests.get(link, headers=headers, timeout=6)
            sub_soup = BeautifulSoup(sub_resp.text, 'html.parser')
            cat_div = sub_soup.find('div', id='mw-normal-catlinks')

            if cat_div:
                cat_text = cat_div.get_text(strip=True).lower()
                if any(k in cat_text for k in keywords_lower):
                    valid_links_new.append(link)
                    print(f"✅ Giữ lại: {link}")
                else:
                    print(f"❌ Loại bỏ: {link}")
                    new_excluded.add(link)
            else:
                print(f"⚠️ Không tìm thấy danh mục: {link}")
                new_excluded.add(link)

        except Exception as e:
            print(f"⚠️ Lỗi khi truy cập {link}: {e}")
            new_excluded.add(link)

        new_crawled.add(link)

    excluded_links.update(new_excluded)
    crawled_links.update(new_crawled)
    save_excluded_links(excluded_links)
    save_crawled_links(crawled_links)

    print(f"💾 Đã lưu {len(new_crawled)} link mới vào danh sách đã crawl.")
    print(f"✅ Tổng số link hợp lệ mới: {len(valid_links_new)}")

    # --- 🔁 Bổ sung: Bao gồm cả link hợp lệ cũ ---
    valid_links_all = set(valid_links_new)

    # Đọc lại tất cả link đã crawl, lọc ra những link hợp lệ cũ (nếu bạn có danh sách riêng lưu hợp lệ)
    # Nếu chưa có, ta có thể coi valid_links_all là tổng hợp hiện tại
    # ⇒ Tức là chỉ thêm những link hợp lệ cũ (đã được lưu trong file khác)
    if os.path.exists("valid_links.json"):
        with open("valid_links.json", "r", encoding="utf-8") as f:
            old_valid = set(json.load(f))
            valid_links_all.update(old_valid)

    # Cập nhật file lưu link hợp lệ tổng hợp
    with open("valid_links.json", "w", encoding="utf-8") as f:
        json.dump(list(valid_links_all), f, ensure_ascii=False, indent=2)

    print(f"📦 Tổng hợp tất cả link hợp lệ: {len(valid_links_all)}")

    return list(valid_links_all)


In [ ]:
def crawl_singer_info(start_urls, depth_limit=DEPTH_LIMIT):
    singers = []
    visited = set()  # tránh trùng lặp
    queue = deque([(url, depth_limit) for url in start_urls])

    while queue:
        url, depth = queue.popleft()  # lấy phần tử đầu (BFS)
        if url in visited or depth <= 0:
            continue
        visited.add(url)

        try:
            print(f"Crawling {url} (depth={depth})...")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Trỏ vào info box
            info_box = soup.find("table", {"class": "infobox"})
            if not info_box:
                continue

            info_rows = info_box.find_all("tr")
            singer_info = {}
            singer_info['depth'] = depth
            singer_info['name'] = soup.find("h1", {"id": "firstHeading"}).get_text(strip=True)

            for row in info_rows:
                header = row.find("th")
                data = row.find("td")
                if header and data:
                    key = header.get_text(strip=True)

                    # --- TRƯỜNG HỢP 1: Có <div class="hlist"> ---
                    hlist_div = data.find("div", {"class": "hlist"})
                    if hlist_div:
                        items = [li.get_text(strip=True) for li in hlist_div.find_all("li")]
                        singer_info[key] = items
                        continue

                    # --- TRƯỜNG HỢP 2: Có <ul> ---
                    ul_tag = data.find("ul")
                    if ul_tag:
                        items = [li.get_text(strip=True) for li in ul_tag.find_all("li")]
                        singer_info[key] = items
                        continue

                    # --- TRƯỜNG HỢP 3: Có <br> ---
                    if data.find("br"):
                        parts = [text.strip() for text in data.stripped_strings]
                        singer_info[key] = [p for p in parts if p]
                    else:
                        value = data.get_text(separator=' ', strip=True)
                        singer_info[key] = value

            # --- Thêm các trường bổ sung ---
            singer_info['năm thành lập'], singer_info['năm tan rã'] = get_years(singer_info.get('Năm hoạt động'))
            singer_info['link'] = url
            singer_info['relations'] = []
            singer_info['awards'] = crawl_singer_award(soup)

            cat_div = soup.find('div', id='mw-normal-catlinks')

            if cat_div:
                cat_text = cat_div.get_text(strip=True).lower()
                # if any cat_text have "nam", add singer_info['giới tính'] = 'nam', "nữ" tương tự
                if "nam" in cat_text:
                    singer_info['giới tính'] = 'nam'
                elif "nữ" in cat_text:
                    singer_info['giới tính'] = 'nữ'

            singers.append(singer_info)

            # --- Thêm các ca sĩ liên quan vào hàng đợi (nếu còn depth) ---
            if depth - 1 > 0:
                colab_links = crawl_valid_links(url)
                singer_info['collaborated_singers'] = colab_links
                for link in colab_links:
                    if link not in visited:
                        queue.append((link, depth - 1))

        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}")

    return singers

In [ ]:
singers = crawl_singer_info(SEED)
print(singers)
#save singer to JSON file


In [ ]:
import json

In [ ]:
# rename fields for better clarity
for singer in singers:
    if "năm thành lập" in singer and "năm tan rã" in singer:
        singer["năm ra mắt"] = singer.pop("năm thành lập", None)
        singer["năm giải nghệ"] = singer.pop("năm tan rã", None)

    singer["Nghề nghiệp khác"] = singer.pop("Nghề nghiệp", None)

    # extract origin (if not present) from other related fields
    if "Nguyên quán" not in singer:
        if "Quê quán" in singer:
            singer["Nguyên quán"] = singer.pop("Quê quán")
        elif "quê quán" in singer:
            singer["Nguyên quán"] = singer.pop("quê quán")
        elif "Nơi sinh" in singer:
            singer["Nguyên quán"] = singer.pop("Nơi sinh")
        elif "nơi sinh" in singer:
            singer["Nguyên quán"] = singer.pop("nơi sinh")
        else:
            singer["Nguyên quán"] = ""

    # --- Chuẩn hóa: chỉ giữ phần trước dấu phẩy (loại bỏ quốc gia) ---
    origin = singer.get("Nguyên quán", "")
    if isinstance(origin, str) and "," in origin:
        # Lấy phần trước dấu phẩy đầu tiên
        singer["Nguyên quán"] = origin.split(",")[0].strip()


In [ ]:
import re
from datetime import datetime 

date_pattern = re.compile(
    r'(?P<day>\d{1,2})[\s\xa0]*tháng[\s\xa0]*(?P<month>\d{1,2})[,，\s\xa0]*(?:năm[\s\xa0]*)?(?P<year>\d{4})',
    re.IGNORECASE
)

for singer in singers:
    if "ngày sinh" in singer:
        s = singer["ngày sinh"]
        if isinstance(s, str):
            s = s.replace('\xa0', ' ')
            match = date_pattern.search(s)
            if match:
                print(f"Tìm thấy: {match.group(0)}")
                day = int(match.group("day"))
                month = int(match.group("month"))
                year = int(match.group("year"))
                try:
                    singer["ngày sinh"] = datetime(year, month, day).date().isoformat()
                except ValueError:
                    singer["ngày sinh"] = ""
                continue

    elif "Sinh" in singer:
        sinh_data = singer.get("Sinh", [])
        if isinstance(sinh_data, list):
            for s in sinh_data:
                if not isinstance(s, str):
                    continue
                s = s.replace('\xa0', ' ')  # 🔧 loại bỏ non-breaking space
                print(f"Kiểm tra chuỗi: {s}")
                match = date_pattern.search(s)
                if match:
                    print(f"Tìm thấy: {match.group(0)}")
                    day = int(match.group("day"))
                    month = int(match.group("month"))
                    year = int(match.group("year"))
                    try:
                        singer["ngày sinh"] = datetime(year, month, day).date().isoformat()
                    except ValueError:
                        singer["ngày sinh"] = ""
                    break
    else:
        singer["ngày sinh"] = ""


In [ ]:

# Mẫu regex xóa mọi nội dung trong ngoặc (), [] cùng khoảng trắng kèm theo
clean_pattern = re.compile(r'\s*(\([^)]*\)|\[[^\]]*\])\s*')

for singer in singers:
    name = singer.get("name", "")
    if not isinstance(name, str):
        continue

    # Xóa nội dung trong ngoặc tròn hoặc ngoặc vuông
    cleaned = re.sub(clean_pattern, '', name).strip()

    # Xóa khoảng trắng dư (nếu có nhiều khoảng trắng)
    cleaned = re.sub(r'\s+', ' ', cleaned)

    singer["name"] = cleaned

    

In [ ]:
import re

# Regex xóa nội dung trong ngoặc () hoặc [] cùng khoảng trắng kèm theo
clean_pattern = re.compile(r'\s*(\([^)]*\)|\[[^\]]*\])\s*')

for singer in singers:
    data = singer.get("Hãng đĩa", None)
    if data is None:
        singer["Hãng đĩa"] = "Độc lập"
        continue

    # Nếu là list
    if isinstance(data, list):
        cleaned_list = []
        for item in data:
            if not isinstance(item, str):
                continue

            # Xóa nội dung trong ngoặc
            cleaned = re.sub(clean_pattern, '', item).strip()
            cleaned = re.sub(r'\s+', ' ', cleaned)

            # Bỏ qua nếu chuỗi rỗng hoặc chỉ chứa ngoặc, số, dấu gạch
            if not cleaned or re.fullmatch(r'[\(\)\[\]\s\d\-–,]*', item.strip()):
                continue

            cleaned_list.append(cleaned)

        singer["Hãng đĩa"] = cleaned_list

    # Nếu là chuỗi
    elif isinstance(data, str):
        cleaned = re.sub(clean_pattern, '', data).strip()
        cleaned = re.sub(r'\s+', ' ', cleaned)

        # Nếu chỉ có ngoặc hoặc rỗng → gán rỗng
        if not cleaned or re.fullmatch(r'[\(\)\[\]\s\d\-–,]*', data.strip()):
            singer["Hãng đĩa"] = ""
        else:
            singer["Hãng đĩa"] = cleaned


In [ ]:
#Doi ten "dong nhac" thanh "the loai"
for singer in singers:
    if 'dong nhac' in singer:
        singer['the loai'] = singer.pop('dong nhac')

In [ ]:
def identify_relationships(singers):
    # Đảm bảo có mảng 'relations'
    for s in singers:
        if 'relations' not in s:
            s['relations'] = []

    for singer in singers:
        for link in singer.get('collaborated_singers', []):
            colab_singer = next((s for s in singers if s['link'] == link), None)
            if not colab_singer:
                continue

            relation_types = []

            # --- Cùng thể loại ---
            singer_genres = set(singer.get("Thể loại", []))
            colab_genres = set(colab_singer.get("Thể loại", []))
            if singer_genres and colab_genres and singer_genres.intersection(colab_genres):
                relation_types.append("same_genre")

            # --- Cộng tác ---
            relation_types.append("collaborated")

            # --- Cùng nguyên quán ---
            if singer.get("Nguyên quán") and colab_singer.get("Nguyên quán"):
                if singer["Nguyên quán"] == colab_singer["Nguyên quán"]:
                    relation_types.append("same_origin")

            # --- Cùng hãng đĩa ---
            label_a = singer.get("Hãng đĩa")
            label_b = colab_singer.get("Hãng đĩa")

            def normalize_label(label):
                """Chuyển về chuỗi thường, loại bỏ khoảng trắng và lấy phần đầu nếu là danh sách"""
                if not label:
                    return ""
                if isinstance(label, list):
                    label = label[0] if label else ""
                return str(label).strip().lower()

            label_a_norm = normalize_label(label_a)
            label_b_norm = normalize_label(label_b)

            if label_a_norm and label_b_norm and label_a_norm == label_b_norm and label_a_norm != "độc lập":
                relation_types.append("same_label")

            # --- Cùng nơi đào tạo ---
            if singer.get("Đào tạo") and colab_singer.get("Đào tạo"):
                if singer["Đào tạo"] == colab_singer["Đào tạo"]:
                    relation_types.append("same_institution")

            # --- Thêm quan hệ vào singer ---
            existing_relation = next((r for r in singer['relations'] if r['singer_link'] == link), None)
            if existing_relation:
                for t in relation_types:
                    if t not in existing_relation['type']:
                        existing_relation['type'].append(t)
            else:
                singer['relations'].append({
                    'singer_link': link,
                    'type': relation_types
                })

            # --- Quan hệ ngược ---
            reverse_relation = next((r for r in colab_singer['relations'] if r['singer_link'] == singer['link']), None)
            if reverse_relation:
                for t in relation_types:
                    if t not in reverse_relation['type']:
                        reverse_relation['type'].append(t)
            else:
                colab_singer['relations'].append({
                    'singer_link': singer['link'],
                    'type': relation_types.copy()
                })


In [ ]:
identify_relationships(singers)

In [ ]:
import json

In [ ]:
with open('../raw_data/singers.json', 'r', encoding='utf-8') as f:
    singers = json.load(f)



In [ ]:
for singer in singers:
    singer.pop("Sinh", None)
    singer.pop("Phối ngẫu", None)
    singer.pop("Nhạc cụ", None)
    singer.pop("Bạn đời", None)
    singer.pop("Con cái", None)
    singer.pop("Giải thưởng", None)
    singer.pop("Website", None)
    singer.pop("collaborated_singers", None)

In [ ]:
with open('../raw_data/singers.json', 'w', encoding='utf-8') as f:
    json.dump(singers, f, ensure_ascii=False, indent=4)